In [ ]:
import pandas as pd
import numpy as np
import requests
files = "business.json"
#gets data from business.json into a dataframe
review_df = pd.read_json (files, lines = True)

In [ ]:
#filters rows that contain nulls
nans = lambda df: df[df.isnull().any(axis=1)]
nulls_df = nans(review_df)
#print(nulls_df)

In [ ]:
#attempt to impute missing address
for addr,name,zipc,longt,lat in zip(nulls_df['address'],nulls_df['name'],nulls_df['postal_code'],nulls_df['longitude'],nulls_df['latitude']):
    if addr == "":
        addr_search = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='+name+'&inputtype=textquery&fields=formatted_address,name,opening_hours&locationbias=circle:2000@'+str(lat)+','+str(longt)+'&key='
        addr_response = requests.get(addr_search)
        print(addr_response.json())
        print(name + ', '+str(lat)+','+str(longt))
        break

In [ ]:
#attempt to impute missing hours
for hour,name,addr in zip(nulls_df['hours'],nulls_df['name'],nulls_df['address']):
    if hour is None:
        if addr != "":
            place_search = 'https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input='+addr+'&inputtype=textquery&key='
            place_response = requests.get(place_search)
            place_id = place_response.json()['candidates'][0]['place_id']
            print(place_id)
            details_search = 'https://maps.googleapis.com/maps/api/place/details/json?place_id='+place_id+'&fields=opening_hours&key='
            details_response = requests.get(details_search)
            place_hours = details_response.json()
            print(place_hours)
            break
            